In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

from scripts.dataPreprocessingPipeline import load_images_for_resnet, unlabeled_image_generator
from scripts.pseudoLabeling import active_semi_sumpervised_learning
from scripts.showImage import show_image, show_images

In [ ]:
X_labeled, y_labeled, labels_list = load_images_for_resnet("../data/labeled")
show_images(X_labeled[:5])

In [ ]:
model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

model.trainable = False

In [ ]:
NUM_CLASSES = len(labels_list)

new_model = models.Sequential([
    model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

In [ ]:
new_model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = new_model.fit(
    X_labeled, y_labeled,
    batch_size=32,
    epochs=18,
    validation_split=0.2,
    shuffle=True
)

Now to semi-supervised + Active Learning part!

In [ ]:
unlabeled_dataset = tf.data.Dataset.from_generator(
    lambda: unlabeled_image_generator("../data/unlabeled", batch_size=32),
    output_signature=(
        tf.TensorSpec(shape=(None,224,224,3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.string)
    )
)


In [ ]:
pseudo_X, pseudo_y, manual_X, manual_y = active_semi_sumpervised_learning(
    new_model,
    unlabeled_dataset,
    threshold=0.9,
    batch_size=32,
)